# Poker Texas Hold'em Coding Dojo

## The problem
This notebook shows a solution to the Coding Dojo "Texas HoldEm" Kata using Python 3.9.
The description of the problem can be found [here](https://codingdojo.org/kata/TexasHoldEm/).

In [36]:
from collections import Counter, defaultdict
from itertools import combinations

## The input
The input of the game (i.e. the hands of the Poker game) were hard-coded as variable: I preferred to focus on solving the problem rather than managing input lines from command line or file.
No checks on the input cards were made, supposing that the given input is well-formatted.

In [37]:
INPUT=[
  "Kc 9s Ks Kd 9d 3c 6d",
  "9c Ah Ks Kd 9d 3c 6d",
  "Ac Qc Ks Kd 9d 3c",
  "9h 5s",
  "4d 2d Ks Kd 9d 3c 6d",
  "7s Ts Ks Kd 9d"
]

## Utilities
I defined two dictionnaries:
- FACES_DICT: to have a sortable value for the card faces
- RANKS: to associate to each rank a sortable value and a human readable string

In [38]:
FACES_DICT={
    "2":2,
    "3":3,
    "4":4,
    "5":5,
    "6":6,
    "7":7,
    "8":8,
    "9":9,
    "T":10,
    "J":11,
    "Q":12,
    "K":13,
    "A":14
    }

RANKS={
 "R_FLUSH":{"value":10, "text":"Royal Flush"},
 "S_FLUSH":{"value":9, "text":"Straight Flush"},
 "FOUR":{"value":8,"text":"Four of a kind"},
 "FULL_H":{"value":7,"text":"Full House"},
 "FLUSH":{"value":6,"text":"Flush"},
 "STRAIGHT":{"value":5, "text":"Straight"},
 "THREE":{"value":4, "text":"Three of a kind"},
 "2_PAIR":{"value":3, "text":"Two pair"},
 "PAIR":{"value":2, "text":"Pair"},
 "HIGH":{"value":1, "text":"High card"},
 "FOLD":{"value":0, "text":"Fold"}
}

## The Python Classes
To solve the problem, I defined four main classes.

### Card
The Card class defines a card of the game, basically constituted by the suit, and the face, which gives the value of the card. 
Using the FACES_DICT dictionnary, an integer was associated to each face in order to be able to sort cards depending on their value.

In [39]:
class Card:
    def __init__(self,twocharstring):
        self.face=twocharstring[0]
        self.suit=twocharstring[1]
        self.sortable_face=FACES_DICT[self.face]
        
    def __str__(self):
        return(self.face+self.suit)
    
    def __repr__(self):
        return str(self)

### Hand
The Hand class defines the hand of a player. It is formed by 7 cards: the 2 own cards, plus the 5 community cards on the table. Only the best 5 of those will be used by the player, while the least 2 cards will be unused.
<br>
This class stores the collection of cards of the player, and a set of methods to assign the best possible rank for the given hand.
<br>
In particular, the "getBestRank" method generates all the possible 5-cards subset for the 7-cards hand, using the itertools.combination method. The Combination class will represent all these possible subsets of cards and will implement methods to find the best rank for each combination.
<br>
The "assignBestRank" method of the Hand class will compare these solutions and find the best 5-cards combination that gives the highest rank to the player's hand.

In [40]:
class Hand:
    def __init__(self,hand_array):
        self.cards=list(map(lambda x: Card(x),hand_array))
        self.best_rank="HIGH"
        self.best_combination=None,
        self.getBestRank()
        
    def assignBestRank(self,new_rank, comb):
        if RANKS[new_rank]["value"]>RANKS[self.best_rank]["value"]:
            self.best_rank=new_rank
            self.best_combination=comb
        
    def getPossibleCombinations(self):
        return combinations(self.cards,5)
    
    def getBestRank(self):
        if self.isFold():
            self.best_combination=Combination(self.cards,[])
            self.best_rank="FOLD"
        else:
            for _c in self.getPossibleCombinations():
                c=Combination(_c,list(set(self.cards)-set(_c)))
                c.getLocalBestRank()
                self.assignBestRank(c.best_rank,c)
            
    def isFold(self):
        return len(self.cards)<7


### Combination
The Combination class defines each 5-cards combination generated by the itertools.combination method within the Hand class. In the constructor of the class, also the 2 unused cards will be passed in order to find the best kicker(s).
<br>
The getLocalBestRank will assign the (local) best rank for each 5-cards combination based on three criteria:
- the 5 cards have, or not have, the same suit (FLUSH)
- the 5 cards contain consecutive faces (STRAIGHT)
- the 5 cards contain groups of identical faces (PAIRS,3-OF-A-KIND,...)

After this process, it will be possible to divide the cards of the player in 3 groups:
- the Ranked cards, i.e. the cards that contribute to the rank
- the Kickers, the cards within the 5-cards best combination, that do not contribute to the rank but that could be used to find a winner in case of tie
- the Unused cards, the 2 cards that will not be taken into consideration for the player's game

In [41]:
class Combination:
    
    def __init__(self,combination, unused_cards):
        self.cards=list(combination),
        self.cards=self.cards[0]
        self.suits=list(map(lambda x: x.suit,self.cards))
        self.faces=list(map(lambda x: x.sortable_face,self.cards))
        self.counter=Counter(self.faces)
        self.best_rank="HIGH"
        self.ranked=[]
        self.kickers=[]
        self.unused=unused_cards

        
    def assignBestRank(self,new_rank):
        if RANKS[new_rank]["value"]>RANKS[self.best_rank]["value"]:
            self.best_rank=new_rank
            
    def isSameSuit(self):
        return self.suits.count(self.suits[0])==len(self.suits)
    
    def hasStraight(self):
        return sorted(self.faces)==list(range(min(self.faces), max(self.faces)+1))
    
    def getLocalBestRank(self):
        # same suits
        if self.isSameSuit():
            if self.hasStraight():
                if sorted(self.faces)==[10,11,12,13,14]:
                    self.assignBestRank("R_FLUSH")
                else:
                    self.assignBestRank("S_FLUSH")
            else:
                self.assignBestRank("FLUSH")
        # different suits
        else:
            if self.hasStraight():
                self.assignBestRank("STRAIGHT")
            else:
                if 4 in self.counter.values():
                    self.assignBestRank("FOUR")
                elif 3 in self.counter.values():
                   
                    if 2 in self.counter.values():
                        self.assignBestRank("FULL_H")
                    else:
                        self.assignBestRank("THREE")
                elif 2 in self.counter.values():
                    if list(self.counter.values()).count(2)==2:
                        self.assignBestRank("2_PAIR")
                    else:
                        self.assignBestRank("PAIR")
                else:
                    self.assignBestRank("HIGH")
                    
        self.defineRanked()
                    
    def defineRanked(self):
        if self.isSameSuit() or self.hasStraight():
            self.ranked=list(sorted(self.cards,key=lambda x:x.sortable_face,reverse=True))
        else:
            if self.best_rank=="HIGH":
                self.ranked.append(list(sorted(self.cards,key=lambda x:x.sortable_face,reverse=True))[0])
                
            else:
                self.ranked=list(sorted(filter(lambda x:self.counter[x.sortable_face]>1, self.cards),key=(lambda x:x.sortable_face), reverse=True))
                
        self.defineKickers()

                    
    def defineKickers(self):
        self.kickers=list(sorted(list(set(self.cards) - set(self.ranked)) + self.unused, key=(lambda x:x.sortable_face), reverse=True))
        self.unused=self.kickers[-2:]
        self.kickers=list(set(self.kickers) - set(self.unused))

### Game
The Game class just initialises the game, taking as input the list of Hands objects, one for each player.
<br>
Then, the "finalRanking" method computes the winner of the game, comparing at first the highest rank for each player (hand). If the hands have the same rank, the values of the Ranked cards should be compared: who has the highest values wins. If the players are still tied, the values Kickers cards should be compared: who has the highest values wins. If there is still a tie, both players are considered as best hands, until they are not being compared to a higher-ranked hand.
<br>
The core of the comparison of tied hands is done by the two methods:
- "compareSequence"
- "compareGroupedSequence"

#### The "compareSequence" method
This methods compares, element-wise, two ordered sequences of values. The algorithm goes on comparing each element of the two arrays until it finds that one element of a sequence is greater than the corresponding element in the second sequence.
<br>
For example, the first elements of the two arrays are compared. If they have the same value, the algorithm goes on to the second element of the two arrays. Otherwise, the algorithm stops and returns an indication (1 or 2) on which array has the highest element.
<br>
If even the last elements of each array are equal, the two sequences are considered tied and 0 is returned.

#### The "compareGroupedSequence" method
This method assumes that every possible sequence of cards can be considered as an ensembe of *n* groups of *m* cards each having the same face.
<br>
For example, a STRAIGHT hand can be considered as an ensemble of 5 groups of 1 card each. Each card (the only one) of the group has a unique face. 
<br>
A FULL_HOUSE hand has 2 groups: one having 3 cards with the same face, the other having 2 cards with the same face.
<br>
So, one can think to build a dictionnary of the occurrences of each face and reverting it, having the cardinality of the groups as keys, and the values of the faces that they represent, as values.
<br>
Since it is possible that two groups with the same cardinality exist, like in the case of a Straight (5 groups of cadinality 1), or in the case of Two Pairs (2 groups of cardinality 2, 1 group of cardinality 1), the values of the dictionnary belonging to the same cardinality are appended to a list, sorted by descending value of the face.
<br>
In that way, in order to compare two tied hands, it is sufficient to consider the cardinalities of each hand in descending order (in the case of Full House, at first consider cardinality 3, then cardinality 2) and compare the corresponding dictionnary values using the "compareSequence" method. If this methods finds that a set of cards has the highest values for that cardinality, that set of cards is the winner.
<br>
The "compareGroupedSequence" is applied both to Ranked cards and Kickers, in order to compare tied hands.

In [42]:
class Game:
    
    def __init__(self, hands):
        self.hands=hands
        self.best_rank="FOLD"
        self.best_hands=[]
        
        
    def finalRanking(self):
        for h in hands:
            if h.best_rank!="FOLD":
                if RANKS[h.best_rank]["value"]>RANKS[self.best_rank]["value"]:
                    self.best_rank=h.best_rank
                    self.best_hands=[]
                    self.best_hands.append(h)
                elif RANKS[h.best_rank]["value"]<RANKS[self.best_rank]["value"]:
                    pass
                else:
                    # tie, compare ranked
                    ranked_a=list(map(lambda x:x.sortable_face,self.best_hands[0].best_combination.ranked))
                    ranked_b=list(map(lambda x:x.sortable_face,h.best_combination.ranked))
                    res=self.compareGroupedSequence(ranked_a, ranked_b)
                    if res==2:
                        self.best_rank=h.best_rank
                        self.best_hands=[]
                        self.best_hands.append(h)
                    elif res==1:
                        self.worst_hands.append(h)
                    else:
                        # tie, compare kickers
                        kickers_a=list(map(lambda x:x.sortable_face,self.best_hands[0].best_combination.kickers))
                        kickers_b=list(map(lambda x:x.sortable_face,h.best_combination.kickers))
                        if len(kickers_a)>0:
                            res2=self.compareGroupedSequence(kickers_a, kickers_b)
                            if res2==2:
                                self.best_rank=h.best_rank
                                self.best_hands=[]
                                self.best_hands.append(h)
                            elif res2==1:
                                pass
                            else:
                                self.best_hands.append(h)
                        else:
                            self.best_hands.append(h)
                    
                
    def compareSequence(self,a,b):
        for i in range(len(a)):
            if a[i]>b[i]:
                return 1
            elif a[i]<b[i]:
                return 2
            else:
                continue
        return 0
    
    
    def compareGroupedSequence(self,a,b):
        a_count=Counter(a)
        b_count=Counter(b)
        
        v = defaultdict(list)
        for key, value in sorted(a_count.items(),reverse=True):
            v[value].append(key)
        a_count=v
        
        v = defaultdict(list)
        for key, value in sorted(b_count.items(),reverse=True):
            v[value].append(key)
        b_count=v
        
        res=0
        for k in sorted(a_count.keys(), reverse=True):
            res=self.compareSequence(a_count[k], b_count[k])
            if res==0:
                continue
            else:
                return res
        return res

## The main
The main of the script instantiate a Hand object for each line of the input and assigns them to a Game object.
<br>
The final ranking is computed and the ranks of the hands are shown on the side of each non-folded hand. a "(Winner)" string is added to the winner hand(s).
The hands of the players are shown considering at first the Ranked cards (ordered by descending value), then the Kickers, and finally the two Unused cards.

In [43]:
hands=[Hand(row.split(" ")) for row in INPUT]
g=Game(hands)
g.finalRanking()
for h in g.hands:
    if h.best_rank!="FOLD":
       if h in g.best_hands:
           print (h.best_combination.ranked+h.best_combination.kickers+h.best_combination.unused, RANKS[h.best_rank]["text"], "(Winner)")
       else:
           print (h.best_combination.ranked+h.best_combination.kickers+h.best_combination.unused, RANKS[h.best_rank]["text"])    
    else:
        print (h.best_combination.cards)

[Kc, Ks, Kd, 9s, 9d, 6d, 3c] Full House (Winner)
[Ks, Kd, 9c, 9d, Ah, 6d, 3c] Two pair
[Ac, Qc, Ks, Kd, 9d, 3c]
[9h, 5s]
[Kd, 9d, 6d, 4d, 2d, Ks, 3c] Flush
[7s, Ts, Ks, Kd, 9d]
